# Model Evaluation Binary Classification

In this notebook will be evaluated the performance of five different basemodels

- XGBoost
- CatBoost
- RandomForest
- LogisticRegression
- MLP

It will be training five different models with a different data split each\
The data split will consist on 2 cities for training, 2 cities for validation and 2 cities for evaluation \
The evaluation will use the mean ROCAU metrics of the five models for each basemodel\
The models will have learning rate fixed to 0.01 and 10 early-stopping rounds (when applied) 

**Results**

| Model           |   ROCAU       |
| -----------     | -----------   |
| XGBoost         |    0.772      |
| Catboost        |               |
| RandomForest    |               |
| LR              |    0.688      |
| MLP             |               |

## 1. Generate Random Data

In [1]:
import sys
sys.path.append('../../src')

import pandas as pd
import numpy as np
import random
from random_data_generator import random_data_generator

#tunning
from tuning import find_best_model

#models
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

#metrics
from sklearn.metrics import roc_auc_score

random.seed(42)

c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = random_data_generator("binary", 5)


## 2. XGBoost

In [3]:
for i in range(0, len(dataset)):
    x_train, y_train, x_val, y_val, x_test, y_test = dataset[i][0], dataset[i][1], dataset[i][2], dataset[i][3], dataset[i][4], dataset[i][5]
    test = find_best_model("xgb", x_train, y_train, x_val, y_val, trials=2)
    break
test

[I 2024-03-08 13:41:37,906] A new study created in memory with name: no-name-739ff4e4-4d03-45a0-9cd9-4f13835256df
Best trial: 0. Best value: 0.729425:  50%|█████     | 1/2 [00:04<00:04,  4.44s/it]

[I 2024-03-08 13:41:42,350] Trial 0 finished with value: 0.7294247128828109 and parameters: {'n_estimators': 117, 'learning_rate': 0.0571227446619491, 'max_depth': 5, 'subsample': 0.41913510639846246, 'colsample_bytree': 0.7141728594210996, 'min_child_weight': 16, 'gamma': 3.3503639844949165}. Best is trial 0 with value: 0.7294247128828109.


Best trial: 0. Best value: 0.729425: 100%|██████████| 2/2 [00:08<00:00,  4.46s/it]

[I 2024-03-08 13:41:46,844] Trial 1 finished with value: 0.7163888342490168 and parameters: {'n_estimators': 85, 'learning_rate': 0.009330783393358112, 'max_depth': 7, 'subsample': 0.8221082903655645, 'colsample_bytree': 0.6413114403884563, 'min_child_weight': 16, 'gamma': 2.4512186782833307}. Best is trial 0 with value: 0.7294247128828109.


FrozenTrial(number=0, state=1, values=[0.7294247128828109], datetime_start=datetime.datetime(2024, 3, 8, 13, 41, 37, 920806), datetime_complete=datetime.datetime(2024, 3, 8, 13, 41, 42, 350056), params={'n_estimators': 117, 'learning_rate': 0.0571227446619491, 'max_depth': 5, 'subsample': 0.41913510639846246, 'colsample_bytree': 0.7141728594210996, 'min_child_weight': 16, 'gamma': 3.3503639844949165}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=150, log=False, low=50, step=1), 'learning_rate': FloatDistribution(high=0.1, log=False, low=0.001, step=None), 'max_depth': IntDistribution(high=8, log=False, low=2, step=1), 'subsample': FloatDistribution(high=0.9, log=False, low=0.3, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.6, step=None), 'min_child_weight': IntDistribution(high=20, log=False, low=1, step=1), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None)}, trial_id=0, valu

In [9]:
xgboost_params = {
    'learning_rate': np.arange(0.001, 0.1, 0.005),
    'max_depth': np.arange(2, 8),
    'n_estimators': np.arange(50, 150, 10),
    'subsample': np.arange(0.3, 0.9, 0.1),
    'colsample_bytree': np.arange(0.6, 1.0, 0.05),
    'gamma': np.arange(0.1, 5, 0.1),
    'early_stopping_rounds': np.arange(5, 15, 5),
    'eval_metric':['auc']
}

scores = []
for i in range(0, len(dataset)):
    x_train, y_train, x_val, y_val, x_test, y_test = dataset[i][0], dataset[i][1], dataset[i][2], dataset[i][3], dataset[i][4], dataset[i][5]
    best_params = find_best_model("xgb", x_train, y_train, x_val, y_val, trials=20)
    xgb_clf = XGBClassifier(**best_params.params, random_state=42).fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)
    y_pred = xgb_clf.predict_proba(x_test)
    scores.append(roc_auc_score(y_test, y_pred[:,1]))
mean =  np.mean(np.array(scores))
print(scores)
print(mean)

[I 2024-03-08 13:51:53,639] A new study created in memory with name: no-name-28d82037-81f5-4cc5-80bb-a4da2ccaeb54
Best trial: 0. Best value: 0.688119:   5%|▌         | 1/20 [00:04<01:18,  4.16s/it]

[I 2024-03-08 13:51:57,799] Trial 0 finished with value: 0.6881188721210227 and parameters: {'n_estimators': 55, 'learning_rate': 0.00515267725220935, 'max_depth': 3, 'subsample': 0.6596766333226011, 'colsample_bytree': 0.894004066304964, 'min_child_weight': 4, 'gamma': 3.1696710648498883}. Best is trial 0 with value: 0.6881188721210227.


Best trial: 1. Best value: 0.729561:  10%|█         | 2/20 [00:07<01:10,  3.92s/it]

[I 2024-03-08 13:52:01,545] Trial 1 finished with value: 0.729560561670316 and parameters: {'n_estimators': 145, 'learning_rate': 0.05306998585672417, 'max_depth': 3, 'subsample': 0.7048951062315595, 'colsample_bytree': 0.9842263753984359, 'min_child_weight': 18, 'gamma': 2.9421223134282366}. Best is trial 1 with value: 0.729560561670316.


Best trial: 1. Best value: 0.729561:  15%|█▌        | 3/20 [00:11<01:01,  3.62s/it]

...

[I 2024-03-08 14:03:24,774] Trial 17 finished with value: 0.723318984560423 and parameters: {'n_estimators': 150, 'learning_rate': 0.08129060476660732, 'max_depth': 6, 'subsample': 0.713721841814498, 'colsample_bytree': 0.897638936856274, 'min_child_weight': 20, 'gamma': 0.5816978771581232}. Best is trial 0 with value: 0.7288001693437234.


Best trial: 0. Best value: 0.7288:  95%|█████████▌| 19/20 [01:57<00:06,  6.99s/it]

[I 2024-03-08 14:03:29,106] Trial 18 finished with value: 0.7151011183018009 and parameters: {'n_estimators': 136, 'learning_rate': 0.0313366584726605, 'max_depth': 6, 'subsample': 0.7078644143480252, 'colsample_bytree': 0.9163042084030021, 'min_child_weight': 1, 'gamma': 1.7475722952163717}. Best is trial 0 with value: 0.7288001693437234.


Best trial: 0. Best value: 0.7288: 100%|██████████| 20/20 [02:04<00:00,  6.21s/it]


[I 2024-03-08 14:03:35,574] Trial 19 finished with value: 0.7279892170856073 and parameters: {'n_estimators': 135, 'learning_rate': 0.05672023611382063, 'max_depth': 8, 'subsample': 0.39479631847829527, 'colsample_bytree': 0.7934954177954924, 'min_child_weight': 1, 'gamma': 1.1787006951275694}. Best is trial 0 with value: 0.7288001693437234.
[0.7908441758672731, 0.752748657647798, 0.8201665511819912, 0.7973922997114995, 0.765066106208158]
0.7852435581233439


## 3. Catboost

In [3]:


scores = []
for i in range(0, len(dataset)):
    x_train, y_train, x_val, y_val, x_test, y_test = dataset[i][0], dataset[i][1], dataset[i][2], dataset[i][3], dataset[i][4], dataset[i][5]
    best_params = find_best_model("cat", x_train, y_train, x_val, y_val, trials=10)
    cat_clf = CatBoostClassifier(**best_params.params, random_seed=42).fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)
    y_pred = cat_clf.predict_proba(x_test)
    scores.append(roc_auc_score(y_test, y_pred[:,1]))
mean =  np.mean(np.array(scores))
print(scores)
print(mean)

[I 2024-03-08 14:59:55,749] A new study created in memory with name: no-name-b1e33199-e379-4299-a95b-b6b964e7cef9
Best trial: 0. Best value: 0.744312:  10%|█         | 1/10 [00:14<02:14, 14.98s/it]

[I 2024-03-08 15:00:10,718] Trial 0 finished with value: 0.7443115797325567 and parameters: {'learning_rate': 0.040983760364399865, 'max_depth': 5, 'n_estimators': 112, 'l2_leaf_reg': 0.8423175647280104}. Best is trial 0 with value: 0.7443115797325567.


Best trial: 0. Best value: 0.744312:  20%|██        | 2/10 [00:28<01:51, 13.91s/it]

[I 2024-03-08 15:00:23,896] Trial 1 finished with value: 0.7325295042906859 and parameters: {'learning_rate': 0.07594022907162075, 'max_depth': 6, 'n_estimators': 67, 'l2_leaf_reg': 0.03534523796260425}. Best is trial 0 with value: 0.7443115797325567.


Best trial: 0. Best value: 0.744312:  30%|███       | 3/10 [00:37<01:23, 11.94s/it]

...

[I 2024-03-08 15:11:05,206] Trial 7 finished with value: 0.6798588833963906 and parameters: {'learning_rate': 0.04096827674166745, 'max_depth': 3, 'n_estimators': 141, 'l2_leaf_reg': 0.587086952969186}. Best is trial 0 with value: 0.7137839521605189.


Best trial: 0. Best value: 0.713784:  90%|█████████ | 9/10 [01:42<00:11, 11.20s/it]

[I 2024-03-08 15:11:19,736] Trial 8 finished with value: 0.6720531236745204 and parameters: {'learning_rate': 0.0042962147827793305, 'max_depth': 5, 'n_estimators': 141, 'l2_leaf_reg': 0.10274313677524527}. Best is trial 0 with value: 0.7137839521605189.


Best trial: 0. Best value: 0.713784: 100%|██████████| 10/10 [01:57<00:00, 11.79s/it]


[I 2024-03-08 15:11:35,290] Trial 9 finished with value: 0.6882297537593767 and parameters: {'learning_rate': 0.04501122820465837, 'max_depth': 7, 'n_estimators': 75, 'l2_leaf_reg': 0.561595318317488}. Best is trial 0 with value: 0.7137839521605189.
[0.8060993280826535, 0.7396892164985717, 0.8274393229696891, 0.7964532523384968, 0.7763066260273186]
0.789197549183346


## 4. Random Forest

In [3]:

scores = []
for i in range(0, len(dataset)):
    x_train, y_train, x_val, y_val, x_test, y_test = dataset[i][0], dataset[i][1], dataset[i][2], dataset[i][3], dataset[i][4], dataset[i][5]
    best_params = find_best_model("random_forest", x_train, y_train, x_val, y_val, trials=5)
    rf_clf = RandomForestClassifier(**best_params.params, random_state=42, verbose=False)
    rf_clf.fit(x_train, y_train)
    y_pred = rf_clf.predict_proba(x_test)
    scores.append(roc_auc_score(y_test, y_pred[:,1]))
mean =  np.mean(np.array(scores))
print(scores)
print(mean)

[I 2024-03-11 13:47:19,067] A new study created in memory with name: no-name-c85f1982-e15b-4580-b146-d243b1edf4f4
Best trial: 0. Best value: 0.731531:  20%|██        | 1/5 [05:06<20:27, 306.76s/it]

[I 2024-03-11 13:52:25,830] Trial 0 finished with value: 0.7315313882334452 and parameters: {'n_estimators': 488, 'max_depth': 4, 'min_samples_split': 10, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 0 with value: 0.7315313882334452.


Best trial: 0. Best value: 0.731531:  40%|████      | 2/5 [29:03<48:34, 971.55s/it]

[I 2024-03-11 14:16:22,726] Trial 1 finished with value: 0.7311896365156021 and parameters: {'n_estimators': 214, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 10, 'bootstrap': False}. Best is trial 0 with value: 0.7315313882334452.


Best trial: 0. Best value: 0.731531:  40%|████      | 2/5 [32:29<48:43, 974.58s/it]


[W 2024-03-11 14:19:48,131] Trial 2 failed with parameters: {'n_estimators': 939, 'max_depth': 2, 'min_samples_split': 8, 'min_samples_leaf': 9, 'bootstrap': True} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\notebooks\Modeling\binary_classfication_comparsion\../../../src\tuning.py", line 96, in <lambda>
    study.optimize(lambda trial: _objective_random_forest(trial, x_train, y_train, x_val, y_val), n_trials=trials, show_progress_bar=True)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\notebooks\Modeling\binary_classfication_comparsion\..

KeyboardInterrupt: 

## 5. Logistic Regression

In [17]:
scores = []
for i in range(0, len(dataset)):
    x_train, y_train, x_val, y_val, x_test, y_test = dataset[i][0], dataset[i][1], dataset[i][2], dataset[i][3], dataset[i][4], dataset[i][5]
    best_params = find_best_model("random_forest", x_train, y_train, x_val, y_val, trials=5)
    lr_clf = LogisticRegression(**best_params, random_state=42, verbose=False)
    lr_clf.fit(x_train, y_train)
    y_pred = lr_clf.predict_proba(x_test)
    scores.append(roc_auc_score(y_test, y_pred[:,1]))
mean =  np.mean(np.array(scores))
print(scores)
print(mean)

c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\caior\OneDrive\Documentos\GitHub\xai-nui-classification\venv\Lib\site-packages\sklearn\linear_model

[0.63691483025621, 0.6219127525835161, 0.4746259835525888, 0.5993430897283265, 0.7113281637915438]
0.608824963982437


## 6. MLP

In [ ]:
scores = []
for i in range(0, len(dataset)):
    x_train, y_train, x_val, y_val, x_test, y_test = dataset[i][0], dataset[i][1], dataset[i][2], dataset[i][3], dataset[i][4], dataset[i][5]
    best_params = find_best_model("mlp", x_train, y_train, x_val, y_val, trials=5)
    lr_clf = MLPClassifier(**best_params, random_state=42, verbose=False)
    lr_clf.fit(x_train, y_train)
    y_pred = lr_clf.predict_proba(x_test)
    scores.append(roc_auc_score(y_test, y_pred[:,1]))
mean =  np.mean(np.array(scores))
print(scores)
print(mean)